<table align="center" width=100%>
    <tr>
        <td width="10%">
            <img src="in_class.png">
        </td>
        <td>
            <div align="center">
                <font color="#21618C" size=8px>
                    <b> Inclass-Lab  <br>(Day 4)
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

## Table of Content

1. **[Chi-Square Test](#chisq)**
2. **[One-way ANOVA](#1way)**
3. **[Equivalent Non-parametric Test](#non-para)**
4. **[Two-way ANOVA](#2way)**

**Import the required libraries**

In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from statsmodels.stats import weightstats as stests

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

import statsmodels.stats.multicomp as mc

!pip install scikit_posthocs
import scikit_posthocs

### Let's begin with some hands-on practice exercises

<a id = "chisq"> </a>
## 1. Chi-Square Test

Use the data available in the CSV file `Employee_Attrition.csv` for questions 1 to 8.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>1. A company in Los Angeles has three functional departments - Research and Development, Sales, and Human Resources. The company claims that the percentage of employees in these 3 departments is 55%, 35% and 10% respectively. Check the company's claim using p-value criteria. Consider a 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [4]:
emp_attrition = pd.read_csv('S7_Employee_Attrition.csv')
emp_attrition.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [12]:
# H0 : Percentage of employees is as per company claims
# H1 : Percentage of employees is different from company claims

emp_attrition['Department'].value_counts()

Research & Development    961
Sales                     446
Human Resources            63
Name: Department, dtype: int64

In [24]:
observed_values    = np.array([961, 446, 63])
n                  = observed_values.sum()
print('n =', n)

expected_values    = np.array([n*0.55, n*0.35, n*0.1])
print('\nObserved values', observed_values)
print('Expected values', expected_values)

chi_square_stat, p_value = stats.chisquare(observed_values, f_exp=expected_values)

print('\nAt 5 %s level of significance, the p-value is', p_value)

print('At 5 %s level of significance, the chi observed is %1.4f' %('%', chi_square_stat))

n = 1470

Observed values [961 446  63]
Expected values [808.5 514.5 147. ]

At 5 %s level of significance, the p-value is 2.2406437256053955e-19
At 5 % level of significance, the chi observed is 85.8847


* In this example, p-value is 2.2406437256053955e-19 and lesser than alpha (0.05)
* So we reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>2. The employees in an IT firm undergo an online assessment survey. The survey reveals that 20% of employees are least satisfied, 18% are fairly satisfied, 30% are moderately satisfied and 32% are highly satisfied with their job. Use a critical value method to test the survey result with 90% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [20]:
# H0 : Percentage of employee satisfaction is as per survey claims
# H1 : Percentage of employee satisfaction is different from survey claims

# JobSatisfaction: 1 = 'Least', 2 = 'Fairly', 3 = 'Moderately', 4 = 'Highly'

emp_attrition['JobSatisfaction'].value_counts()

4    459
3    442
1    289
2    280
Name: JobSatisfaction, dtype: int64

In [21]:
observed_values    = np.array([289, 280, 442, 459])
n                  = observed_values.sum()
print('n =', n)

expected_values    = np.array([n*0.2, n*0.18, n*0.3, n*0.32])
print('\nObserved values', observed_values)
print('Expected values', expected_values)

chi_square_stat, p_value = stats.chisquare(observed_values, f_exp=expected_values)

print('\nAt 5 %s level of significance, the p-value is', p_value)

print('At 5 %s level of significance, the chi observed is %1.4f' %('%', chi_square_stat))

n = 1470

Observed values [289 280 442 459]
Expected values [294.  264.6 441.  470.4]

At 5 %s level of significance, the p-value is 0.7386828586247451
At 5 % level of significance, the chi observed is 1.2599


In [22]:
# calculate the χ2-value for 90% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'

chi2_val = np.abs(round(stats.chi2.isf(q = 0.1, df = 3), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 6.2514


* In this example, test statistic (1.2599) is lesser than Chi-crit (6.2514) 
* Test statistic falls to left of Chi-crit i.e. it falls in acceptance region of H0
* p-value is  0.7386 and greater than alpha (0.1)
* So we fail to reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>3. The company claims that in the organization, 68% of employees are there with rare travelling, 20% with frequent travelling and 12% of employees does not need to travel for business. Check if the given data fits with the company's claimed distribution. Use the p-value technique with 99% confidence. </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [23]:
# H0 : Percentage of travel is as per company claims
# H1 : Percentage of travel is different from company claims

emp_attrition['BusinessTravel'].value_counts()

Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: BusinessTravel, dtype: int64

In [25]:
observed_values    = np.array([1043, 277, 150])
n                  = observed_values.sum()
print('n =', n)

expected_values    = np.array([n*0.68, n*0.2, n*0.12])
print('\nObserved values', observed_values)
print('Expected values', expected_values)

chi_square_stat, p_value = stats.chisquare(observed_values, f_exp=expected_values)

print('\nAt 5 %s level of significance, the p-value is', p_value)

print('At 5 %s level of significance, the chi observed is %1.7f' %('%', chi_square_stat))

n = 1470

Observed values [1043  277  150]
Expected values [999.6 294.  176.4]

At 5 %s level of significance, the p-value is 0.0330688454314459
At 5 % level of significance, the chi observed is 6.8183273


* In this example, p-value is 0.0330688454314459 and greater than alpha (0.01)
* So we fail to reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>4. Check whether travelling for work depends upon the job role of an employee. Use p-value criteria to test the dependence with 99% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [37]:
# H0 : Travelling for business is independent of job role
# H1 : Travelling for business is dependent on job role

# use 'crosstab()' to create a table for each job role level and business travel count 
table = pd.crosstab(emp_attrition['JobRole'], emp_attrition['BusinessTravel'])
print(table)

# observed values  
observed_value = table.values
observed_value

BusinessTravel             Non-Travel  Travel_Frequently  Travel_Rarely
JobRole                                                                
Healthcare Representative          15                 26             90
Human Resources                     4                 10             38
Laboratory Technician              28                 51            180
Manager                            12                 13             77
Manufacturing Director             13                 29            103
Research Director                   6                 12             62
Research Scientist                 28                 54            210
Sales Executive                    39                 59            228
Sales Representative                5                 23             55


array([[ 15,  26,  90],
       [  4,  10,  38],
       [ 28,  51, 180],
       [ 12,  13,  77],
       [ 13,  29, 103],
       [  6,  12,  62],
       [ 28,  54, 210],
       [ 39,  59, 228],
       [  5,  23,  55]], dtype=int64)

In [33]:
# calculate the χ2-value for 99% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'

# df = (r - 1) * (c - 1)
# df = 8 * 2 = 16

chi2_val = np.abs(round(stats.chi2.isf(q = 0.01, df = 16), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 31.9999


In [41]:
# use the 'chi2_contingency()' to check the independence of variables

test_stat, p, dof, expected_value = stats.chi2_contingency(observed = observed_value, correction = False)

print("Test statistic:", test_stat)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("\nExpected values:", expected_value)

Test statistic: 11.987695596739206
p-value: 0.7448263418408124
Degrees of freedom: 16

Expected values: [[ 13.36734694  24.68503401  92.94761905]
 [  5.30612245   9.79863946  36.8952381 ]
 [ 26.42857143  48.8047619  183.76666667]
 [ 10.40816327  19.22040816  72.37142857]
 [ 14.79591837  27.32312925 102.88095238]
 [  8.16326531  15.07482993  56.76190476]
 [ 29.79591837  55.02312925 207.18095238]
 [ 33.26530612  61.42993197 231.3047619 ]
 [  8.46938776  15.64013605  58.89047619]]


* In this example, p-value is 0.7448263418408124 and greater than alpha (0.01)
* Test statistic (11.9876) is less than Chi-crit (31.9999) i.e. it falls in acceptance region of H0
* So we fail to reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>5. Is there any relationship between the attrition of an employee and his/her marital status? Use the critical value technique to test the relationship with 95% confidence. </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [42]:
# H0 : Employee attrition is independent of marital status
# H1 : Employee attrition is dependent on marital status

# use 'crosstab()' to create a table for each job role level and business travel count 
table = pd.crosstab(emp_attrition['MaritalStatus'], emp_attrition['Attrition'])
print(table)

# observed values  
observed_value = table.values
observed_value

Attrition       No  Yes
MaritalStatus          
Divorced       294   33
Married        589   84
Single         350  120


array([[294,  33],
       [589,  84],
       [350, 120]], dtype=int64)

In [43]:
# calculate the χ2-value for 99% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'

# df = (r - 1) * (c - 1)
# df = 2 * 1 = 2

chi2_val = np.abs(round(stats.chi2.isf(q = 0.05, df = 2), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 5.9915


In [44]:
# use the 'chi2_contingency()' to check the independence of variables

test_stat, p, dof, expected_value = stats.chi2_contingency(observed = observed_value, correction = False)

print("Test statistic:", test_stat)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("\nExpected values:", expected_value)

Test statistic: 46.163676540848705
p-value: 9.45551106034083e-11
Degrees of freedom: 2

Expected values: [[274.27959184  52.72040816]
 [564.49591837 108.50408163]
 [394.2244898   75.7755102 ]]


* In this example, test statistic (46.1636) is greater than Chi-crit (5.9915) i.e. it falls in rejection region of H0 
* p-value is 9.45551106034083e-11 and lesser than alpha (0.05) 
* So we reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>6. Check whether travelling for the business depends on the gender of an employee. Use the p-value technique to test the claim with 90% confidence. </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [45]:
# H0 : Travelling for business is independent of gender
# H1 : Travelling for business is dependent on gender

# use 'crosstab()' to create a table for each job role level and business travel count 
table = pd.crosstab(emp_attrition['Gender'], emp_attrition['BusinessTravel'])
print(table)

# observed values  
observed_value = table.values
observed_value

BusinessTravel  Non-Travel  Travel_Frequently  Travel_Rarely
Gender                                                      
Female                  49                117            422
Male                   101                160            621


array([[ 49, 117, 422],
       [101, 160, 621]], dtype=int64)

In [46]:
# calculate the χ2-value for 99% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'

# df = (r - 1) * (c - 1)
# df = 1 * 2 = 2

chi2_val = np.abs(round(stats.chi2.isf(q = 0.1, df = 2), 4))

print('Critical value for chi-square test:', chi2_val)

Critical value for chi-square test: 4.6052


In [47]:
# use the 'chi2_contingency()' to check the independence of variables

test_stat, p, dof, expected_value = stats.chi2_contingency(observed = observed_value, correction = False)

print("Test statistic:", test_stat)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("\nExpected values:", expected_value)

Test statistic: 4.031372310350092
p-value: 0.13322895625828154
Degrees of freedom: 2

Expected values: [[ 60.  110.8 417.2]
 [ 90.  166.2 625.8]]


* In this example, p-value is 0.13322895625828154 and greater than alpha (0.1)
* Test statistic (4.03137) is less than Chi-crit (4.6052) i.e. it falls in acceptance region of H0
* So we fail to reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>7. The economic forum claims that the variance in the percentage salary hike is more the 12% in Manhattan. To test the forum's claim use the data obtained from a telecom company in Manhattan. Use a critical value method to test the claim with 90% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [20]:
# H0 : var <= 12
# H1 : var > 12

# alpha = 0.1

# hypothesized variance
sig_2 = 12

# population size
n = len(emp_attrition)

# degrees of freedom
print('Degrees of freedom:', n-1)

# population mean
pop_mean = np.mean(emp_attrition['PercentSalaryHike'])
print('Population mean:', pop_mean)

Degrees of freedom: 1469
Population mean: 15.209523809523809


In [21]:
# calculate the χ2-value for 95% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'

chi2_val_right = np.abs(round(stats.chi2.isf(q = 1-0.1, df = n-1), 4))

print('Critical value for right-tail chi-square test:', chi2_val_right)

Critical value for right-tail chi-square test: 1399.978


In [22]:
# calculate the test statistic
chi_test = (np.sum((emp_attrition['PercentSalaryHike'] - pop_mean)**2)) / sig_2

# print the test statistic
print('Test Statistic:', chi_test)

Test Statistic: 1639.788888888889


In [24]:
# calculate P(X > chi_test)
p_val = stats.chi2.sf(x = chi_test, df = n-1)

# print the p-value
print('p-value:', p_val)

p-value: 0.0011494196273865566


* In this example, test statistic (1639.78888) is greater than Chi-crit (1399.978) for upper-tailed test
* Test statistic falls to right side of Chi-crit i.e. it falls in rejection region of H0
* p-value is 0.00114941 and lesser than alpha (0.1)
* So we reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>8. The worker's association claims that the distance travelled by each employee to reach the office has a variance of less than 15 miles. Use a critical value method to test the claim with 95% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [10]:
# H0 : var >= 15
# H1 : var < 15

# alpha = 0.05

# hypothesized variance
sig_2 = 15

# population size
n = len(emp_attrition)

# degrees of freedom
print('Degrees of freedom:', n-1)

# population mean
pop_mean = np.mean(emp_attrition['DistanceFromHome'])
print('Population mean:', pop_mean)

Degrees of freedom: 1469
Population mean: 9.19251700680272


In [11]:
# calculate the χ2-value for 95% of confidence level
# use 'stats.chi2.ppf()' to find the χ2-value corresponding to the lower tail probability 'q'

chi2_val_left = np.abs(round(stats.chi2.ppf(q = 0.05, df = n-1), 4))

print('Critical value for left-tail chi-square test:', chi2_val_left)

Critical value for left-tail chi-square test: 1380.9949


In [12]:
# calculate the test statistic
chi_test = (np.sum((emp_attrition['DistanceFromHome'] - pop_mean)**2)) / sig_2

# print the test statistic
print('Test Statistic:', chi_test)

Test Statistic: 6436.301179138322


In [15]:
# calculate P(X < chi_test)
p_val = stats.chi2.cdf(x = chi_test, df = n-1)

# print the p-value
print('p-value:', p_val)

p-value: 1.0


* In this example, test statistic (6436.30117) is greater than Chi-crit (1380.9949) for lower tailed test
* Test statistic falls to right side of Chi-crit i.e. it falls in acceptance region of H0
* p-value is 1 and greater than alpha (0.05)
* So we fail to reject the null hypothesis

<a id = "1way"> </a>
## 2. One-way ANOVA

Use the data available in the CSV file `sales_emp.csv` for questions 9 to 13.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>9. Check whether we can use the given dataset to study the equality of average monthly income of sales executives with a different education background in the company. Use a p-value technique to test at a 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [2]:
sales_emp = pd.read_csv('S7_sales_emp.csv')
sales_emp.head()

,Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,36,Travel_Rarely,1218,9,4,Life Sciences,1,27,3,82,...,2,80,0,10,4,3,5,3,0,3
1,39,Travel_Rarely,895,5,3,Technical Degree,1,42,4,56,...,3,80,1,19,6,4,1,0,0,0
2,50,Travel_Rarely,869,3,2,Marketing,1,47,1,86,...,3,80,0,3,2,3,3,2,0,2
3,46,Travel_Rarely,669,9,2,Medical,1,118,3,64,...,4,80,0,9,3,3,9,8,4,7
4,25,Non-Travel,240,5,3,Marketing,1,142,3,46,...,4,80,0,6,1,3,6,4,0,3


In [24]:
# H0: The average monthly income of sales executives is same
# H1: At least one set of sales executives with different education background has a different monthly income

# unique education background in the data
sales_emp['EducationField'].unique()

array(['Life Sciences', 'Technical Degree', 'Marketing', 'Medical',
       'Other'], dtype=object)

There are total 5 unique education fields in the dataset.

In [25]:
# given data
gr_lf = sales_emp[sales_emp['EducationField'] == 'Life Sciences']['MonthlyIncome']
gr_td = sales_emp[sales_emp['EducationField'] == 'Technical Degree']['MonthlyIncome']
gr_mar = sales_emp[sales_emp['EducationField'] == 'Marketing']['MonthlyIncome']
gr_med = sales_emp[sales_emp['EducationField'] == 'Medical']['MonthlyIncome']
gr_oth = sales_emp[sales_emp['EducationField'] == 'Other']['MonthlyIncome']

In [26]:
# perform Shapiro-Wilk test to test the normality
stat, p_value = stats.shapiro(sales_emp['MonthlyIncome'])

# print the p-value
print('p-value:', p_value)

p-value: 0.05732710659503937


From the above result, we can see that the p-value is greater than 0.05, thus we can say that the average monthly income of sales executives from each group are normally distributed. Thus the assumption of normality is satisfied.

In [27]:
# perform Levene's test for the equality of variances 
stat, p_value = stats.levene(gr_lf, gr_td, gr_mar, gr_med, gr_oth)

# print the p-value 
print('p-value:', p_value)

p-value: 0.23476859109336565


From the above result, we can see that the p-value is greater than 0.05, thus we can say that the population variances are equal for all the samples.

* Assumption of normality is satisfied by Shapiro-Wilk test
* Assumption of equality of variance is satisfied by Levene's test
* So we can use the dataset to study the equality of average monthly income

In [28]:
# obtain number of unique education backgrounds
t = sales_emp['EducationField'].nunique()
print('t:', t)

# consider the total number of employees in all education backgrounds
N = sales_emp['EducationField'].value_counts().sum()
print('N:', N)

t: 5
N: 54


In [29]:
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'

f = np.abs(round(stats.f.isf(q = 0.05, dfn = t-1, dfd = N-t), 4))

print('Critical value for F-test:', f)

Critical value for F-test: 2.5611


In [30]:
# perform one-way ANOVA
# pass the given data
f_test, p_val = stats.f_oneway(gr_lf, gr_td, gr_mar, gr_med, gr_oth)

# print the test statistic and corresponding p-value
print('Test statistic:', f_test)
print('p-value:', p_val)

Test statistic: 0.9546031924978221
p-value: 0.4407865128409415


* In this example, p-value (0.4407) is greater than alpha (0.05)
* So we fail to reject the null hypothesis

 <table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>10. Use the sales employees' dataset to test whether the average monthly income of sales executives with different education background is equal or not. Use a critical value method with 95% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [31]:
# H0: The average monthly income of sales executives is same
# H1: At least one set of sales executives with different education background has a different monthly income

# given data
gr_lf = sales_emp[sales_emp['EducationField'] == 'Life Sciences']['MonthlyIncome']
gr_td = sales_emp[sales_emp['EducationField'] == 'Technical Degree']['MonthlyIncome']
gr_mar = sales_emp[sales_emp['EducationField'] == 'Marketing']['MonthlyIncome']
gr_med = sales_emp[sales_emp['EducationField'] == 'Medical']['MonthlyIncome']
gr_oth = sales_emp[sales_emp['EducationField'] == 'Other']['MonthlyIncome']

In [32]:
# obtain number of unique education backgrounds
t = sales_emp['EducationField'].nunique()
print('t:', t)

# consider the total number of employees in all education backgrounds
N = sales_emp['EducationField'].value_counts().sum()
print('N:', N)

t: 5
N: 54


In [33]:
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'

f = np.abs(round(stats.f.isf(q = 0.05, dfn = t-1, dfd = N-t), 4))

print('Critical value for F-test:', f)

Critical value for F-test: 2.5611


In [34]:
# perform one-way ANOVA
# pass the given data
f_test, p_val = stats.f_oneway(gr_lf, gr_td, gr_mar, gr_med, gr_oth)

# print the test statistic and corresponding p-value
print('Test statistic:', f_test)
print('p-value:', p_val)

Test statistic: 0.9546031924978221
p-value: 0.4407865128409415


* In this example, test statistic (0.4407) is lesser than Chi-crit (2.5611)
* Test statistic falls to the left of Chi-crit i.e. it falls in acceptance region of H0
* So we fail to reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>11. Can we use the given data of sales executives to check the equality of the average daily rate for different types of business travellers? Use a p-value technique to test at a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [35]:
sales_emp.head()

,Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,36,Travel_Rarely,1218,9,4,Life Sciences,1,27,3,82,...,2,80,0,10,4,3,5,3,0,3
1,39,Travel_Rarely,895,5,3,Technical Degree,1,42,4,56,...,3,80,1,19,6,4,1,0,0,0
2,50,Travel_Rarely,869,3,2,Marketing,1,47,1,86,...,3,80,0,3,2,3,3,2,0,2
3,46,Travel_Rarely,669,9,2,Medical,1,118,3,64,...,4,80,0,9,3,3,9,8,4,7
4,25,Non-Travel,240,5,3,Marketing,1,142,3,46,...,4,80,0,6,1,3,6,4,0,3


In [36]:
# H0: The average daily rate for different types of business travellers is same
# H1: The average daily rate for different types of business travellers is different

# unique education background in the data
sales_emp['BusinessTravel'].unique()

array(['Travel_Rarely', 'Non-Travel', 'Travel_Frequently'], dtype=object)

There are total 3 unique types of business travel in the dataset.

In [3]:
# H0: The average daily rate for different types of business travellers is same
# H1: The average daily rate for different types of business travellers is different

# given data
gr_tr = sales_emp[sales_emp['BusinessTravel'] == 'Travel_Rarely']['DailyRate']
gr_nt = sales_emp[sales_emp['BusinessTravel'] == 'Non-Travel']['DailyRate']
gr_tf = sales_emp[sales_emp['BusinessTravel'] == 'Travel_Frequently']['DailyRate']

In [38]:
# perform Shapiro-Wilk test to test the normality
stat, p_value = stats.shapiro(sales_emp['DailyRate'])

# print the p-value
print('p-value:', p_value)

p-value: 0.040250398218631744


From the above result, we can see that the p-value is greater than 0.01, thus we can say that the average daily rate from each group are normally distributed. Thus the assumption of normality is satisfied.

In [39]:
# perform Levene's test for the equality of variances 
stat, p_value = stats.levene(gr_tr, gr_nt, gr_tf)

# print the p-value 
print('p-value:', p_value)

p-value: 0.23137904227432696


From the above result, we can see that the p-value is greater than 0.01, thus we can say that the population variances are equal for all the samples.

* Assumption of normality is satisfied by Shapiro-Wilk test
* Assumption of equality of variance is satisfied by Levene's test
* So we can use the dataset to study the equality of average daily rate

In [4]:
# obtain number of unique types of business travellers
t = sales_emp['BusinessTravel'].nunique()
print('t:', t)

# consider the total number of employees in all types of business travel
N = sales_emp['BusinessTravel'].value_counts().sum()
print('N:', N)

t: 3
N: 54


In [5]:
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'

f = np.abs(round(stats.f.isf(q = 0.01, dfn = t-1, dfd = N-t), 4))

print('Critical value for F-test:', f)

Critical value for F-test: 5.0472


In [42]:
# perform one-way ANOVA
# pass the given data
f_test, p_val = stats.f_oneway(gr_tr, gr_nt, gr_tf)

# print the test statistic and corresponding p-value
print('Test statistic:', f_test)
print('p-value:', p_val)

Test statistic: 15.815759872412409
p-value: 4.527131533131016e-06


* In this example, p-value (4.5271e-06) is lesser than alpha (0.01)
* Test statistic (15.8157) is greater than Chi-crit (5.0472) i.e. it falls in rejection region of H0
* So we reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>12. Use the parametric test to check the equality of the average daily rate for all the types of business travelling. Use a p-value technique to test the data with 99% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [8]:
# H0: The average daily rate for different types of business travellers is same
# H1: The average daily rate for different types of business travellers is different

# perform one-way ANOVA

# fit an ols model on the datafreme 'sales_emp' 
# use 'Q()' to quote the variable name 
# use 'fit()' to fit the linear model
test = ols('DailyRate ~ Q("BusinessTravel")', sales_emp).fit()

# create table for one-way ANOVA test
# pass the linear model 'test'
# 'typ = 1' performs two-way ANOVA
anova_1 = anova_lm(test, typ = 1)

# print the table
anova_1

,df,sum_sq,mean_sq,F,PR(>F)
"Q(""BusinessTravel"")",2.0,3.755400e+06,1.877700e+06,15.81576,0.000005
Residual,51.0,6.054890e+06,1.187233e+05,NaN,NaN


* In this example, p-value (4.5271e-06) is lesser than alpha (0.01)
* So we reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>13. Find the types of business travel for which the average daily rate is different. Use a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [49]:
# in above one-way ANOVA, we rejected the null hypothesis. 
# We conclude that atleast one type of business travellers has different avg daily rate
# The post-hoc test or multi comparison test is used to identify this different group of business travellers
# perform tukey's range test to compare the average daily rate for different types of business travellers

# H0: The average daily rate for different types of business travellers is same
# H1: The average daily rate for different types of business travellers is different

comp = mc.MultiComparison(data = sales_emp['DailyRate'], groups = sales_emp['BusinessTravel'])

# tukey's range test
post_hoc = comp.tukeyhsd()

# print the summary table
post_hoc.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
Non-Travel,Travel_Frequently,639.7778,0.001,362.5256,917.03,True
Non-Travel,Travel_Rarely,242.6667,0.0972,-34.5855,519.9189,False
Travel_Frequently,Travel_Rarely,-397.1111,0.0031,-674.3633,-119.8589,True


The reject=False for pairs (Non-Travel, Travel_Rarely) denotes that we fail to reject the null hypothesis; and conclude that the average daily rate due to Non-Travel and Travel_Rarely are same.

The average daily rate is different for (Non-Travel, Travel_Frequently) and (Travel_Frequently, Travel_Rarely)

The values in the columns lower and upper represent the lower and upper bound of the 95% confidence interval for the mean difference.

<a id= "non-para"></a>
## 3. Equivalent Non-parametric Test

Use the data available in the CSV file `Employee_Attrition.csv` for questions 14 to 18.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>14. Can we use the non-parametric test to study the equality of average monthly income for all departments in the company? Use a p-value technique to test at 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [52]:
# H0 : The average monthly income for all departments is the same
# H1 : The average monthly income for all departments is different

emp_attrition = pd.read_csv('S7_Employee_Attrition.csv')
emp_attrition.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [51]:
# perform Shapiro-Wilk test to test the normality
stat, p_value = stats.shapiro(emp_attrition['MonthlyIncome'])

# print the p-value
print('p-value:', p_value)

p-value: 4.4031748323132235e-37


From the above result, we can see that the p-value is less than 0.01, thus we can say that the average monthly income for all departments in the company is not normally distributed. This does not satisfy the assumption of normality for ANOVA.

Thus we can not apply one-way ANOVA to test the equality of average monthly income. We need to use the non-parametric equivalent Kruskal-Wallis H test

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                        <b>15. Check whether the monthly income for all the departments in a company is the same or not. Test using the critical value method at a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [57]:
# H0 : The average monthly income for all departments is the same
# H1 : The average monthly income for all departments is different

# obtain number of unique types of departments
t = emp_attrition['Department'].nunique()
print('t:', t)

# unique education background in the data
print(emp_attrition['Department'].unique())

# calculate the χ2-value for 99% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'
chi2_val = np.abs(round(stats.chi2.isf(q = 0.01, df = t-1), 4))

print('\nCritical value for chi-square test:', chi2_val)

t: 3
['Sales' 'Research & Development' 'Human Resources']

Critical value for chi-square test: 9.2103


In [58]:
inc_sales = emp_attrition[emp_attrition['Department'] == 'Sales']['MonthlyIncome']
inc_rnd = emp_attrition[emp_attrition['Department'] == 'Research & Development']['MonthlyIncome']
inc_hr = emp_attrition[emp_attrition['Department'] == 'Human Resources']['MonthlyIncome']

# perform kruskal-wallis H test
test_stat, p_val = stats.kruskal(inc_sales, inc_rnd, inc_hr)

# print the test statistic and corresponding p-value
print('Test statistic:', test_stat)
print('p-value:', p_val)

Test statistic: 42.61858855841987
p-value: 5.565332051212746e-10


* In this example, test statistic (42.6185) is greater than Chi-crit (9.2103)
* Test statistic falls to the right of Chi-crit i.e. it falls in rejection region of H0
* p-value (5.5653e-10) is less than alpha (0.01)
* So we reject the null hypothesis

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>16. Is it possible to identify the department with different average monthly income? If yes, perform the test at a 1% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

Since in the above question, Kruskal-Wallis H test rejects the null hypothesis; we conclude that at least one department has a average monthly income. The test does not distinguish a department with the different average monthly income. The post-hoc test is used to identify the department. Thus we use the conover test as a post-hoc test.

In [64]:
# perform the conover test
scikit_posthocs.posthoc_conover(a = emp_attrition, val_col = 'MonthlyIncome', group_col = 'Department')

,Human Resources,Research & Development,Sales
Human Resources,1.000000,7.431216e-01,2.076733e-03
Research & Development,0.743122,1.000000e+00,1.084486e-10
Sales,0.002077,1.084486e-10,1.000000e+00


For α = 0.01, the p-value for pairs (HR, Sales) and (R & D, Sales) is less than 0.01. Thus we can conclude that there is a difference in the in the average monthly income for the pairs of departments (HR, Sales) and (R & D, Sales).

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>17. Check whether we can perform the Kruskal-Wallis H test to test the equality of the average experience of employees in all the departments. Consider a 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [65]:
# H0 : The average experience (TotalWorkingYears) for all departments is the same
# H1 : The average experience (TotalWorkingYears) for all departments is different

# perform Shapiro-Wilk test to test the normality
stat, p_value = stats.shapiro(emp_attrition['TotalWorkingYears'])

# print the p-value
print('p-value:', p_value)

p-value: 5.62833706762415e-29


From the above result, we can see that the p-value is less than 0.05, thus we can say that the average experience of employees in all the departments is not normally distributed. This does not satisfy the assumption of normality for ANOVA.

Thus we can not apply one-way ANOVA to test the equality of average monthly income. We can perform the non-parametric equivalent Kruskal-Wallis H test

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                        <b>18. Perform a non-parametric test to test the equality of the average experience of employees in all the departments. Use the p-value technique to perform the test with 95% confidence.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [66]:
# H0 : The average experience (TotalWorkingYears) for all departments is the same
# H1 : The average experience (TotalWorkingYears) for all departments is different

# obtain number of unique types of departments
t = emp_attrition['Department'].nunique()
print('t:', t)

# unique education background in the data
print(emp_attrition['Department'].unique())

# calculate the χ2-value for 99% of confidence level
# use 'stats.chi2.isf()' to find the χ2-value corresponding to the upper tail probability 'q'
chi2_val = np.abs(round(stats.chi2.isf(q = 0.05, df = t-1), 4))

print('\nCritical value for chi-square test:', chi2_val)

t: 3
['Sales' 'Research & Development' 'Human Resources']

Critical value for chi-square test: 5.9915


In [67]:
exp_sales = emp_attrition[emp_attrition['Department'] == 'Sales']['TotalWorkingYears']
exp_rnd = emp_attrition[emp_attrition['Department'] == 'Research & Development']['TotalWorkingYears']
exp_hr = emp_attrition[emp_attrition['Department'] == 'Human Resources']['TotalWorkingYears']

# perform kruskal-wallis H test
test_stat, p_val = stats.kruskal(exp_sales, exp_rnd, exp_hr)

# print the test statistic and corresponding p-value
print('Test statistic:', test_stat)
print('p-value:', p_val)

Test statistic: 0.14222005425570283
p-value: 0.9313594116888158


* In this example, p-value (0.9313) is greater than alpha (0.05)
* Test statistic (0.1422) is lesser than Chi-crit (5.9915)
* Test statistic falls to the left of Chi-crit i.e. it falls in acceptance region of H0
* So we fail to reject the null hypothesis

<a id= "2way"></a>
## 4. Two-way ANOVA

Use the data available in the CSV file `sales_emp.csv` for questions 19 and 20.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>19. Check whether the monthly income of employee depends on the education field and job involvement of the employee. Use the p-value technique with 5% level of significance.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [90]:
sales_emp = pd.read_csv('S7_sales_emp.csv')
sales_emp.head()

,Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,36,Travel_Rarely,1218,9,4,Life Sciences,1,27,3,82,...,2,80,0,10,4,3,5,3,0,3
1,39,Travel_Rarely,895,5,3,Technical Degree,1,42,4,56,...,3,80,1,19,6,4,1,0,0,0
2,50,Travel_Rarely,869,3,2,Marketing,1,47,1,86,...,3,80,0,3,2,3,3,2,0,2
3,46,Travel_Rarely,669,9,2,Medical,1,118,3,64,...,4,80,0,9,3,3,9,8,4,7
4,25,Non-Travel,240,5,3,Marketing,1,142,3,46,...,4,80,0,6,1,3,6,4,0,3


In [91]:
sales_emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       54 non-null     int64 
 1   BusinessTravel            54 non-null     object
 2   DailyRate                 54 non-null     int64 
 3   DistanceFromHome          54 non-null     int64 
 4   Education                 54 non-null     int64 
 5   EducationField            54 non-null     object
 6   EmployeeCount             54 non-null     int64 
 7   EmployeeNumber            54 non-null     int64 
 8   EnvironmentSatisfaction   54 non-null     int64 
 9   HourlyRate                54 non-null     int64 
 10  JobInvolvement            54 non-null     int64 
 11  JobLevel                  54 non-null     int64 
 12  JobSatisfaction           54 non-null     int64 
 13  MaritalStatus             54 non-null     object
 14  MonthlyIncome             54

In [92]:
# since default Dtype of JobInvolvement is int64, we first need to make it categorical/nominal 
sales_emp['JobInvolvement'] = sales_emp['JobInvolvement'].astype(object)

The two-way ANOVA can be used to test the effect due to the two nominal variables on a numeric variable.

The null and alternative hypothesis for education field:
* H01 : The monthly income is independent of education field 
* H11 : The monthly income is dependent on education field

The null and alternative hypothesis for job involvement:
* H02 : The monthly income is independent of job involvement 
* H12 : The monthly income is dependent on job involvement

In [69]:
# perform Shapiro-Wilk test to test the normality
stat, p_value = stats.shapiro(sales_emp['MonthlyIncome'])

# print the p-value
print('p-value:', p_value)

p-value: 0.05732710659503937


From the above result, we can see that the p-value is greater than 0.05, thus we can say that the monthly income from different groups of education fields and job involvment are normally distributed. Thus the assumption of normality is satisfied.

In [93]:
# obtain number of unique education fields (treatments)
t = sales_emp['EducationField'].nunique()
print('t:', t)

# obtain number of unique job involvement levels (blocks)
b = sales_emp['JobInvolvement'].nunique()
print('b:', b)

# consider the total number of employees in all education fields
N = sales_emp['EducationField'].value_counts().sum()
print('N:', N)

t: 5
b: 4
N: 54


In [97]:
# degrees of freedom for treatment df_tr = [t-1, (t-1)(b-1)] = (4, 12)
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'

f = np.abs(round(stats.f.isf(q = 0.05, dfn = 4, dfd = 12), 4))
print('Critical value (treatment - EducationField) for F-test:', f)

Critical value (treatment - EducationField) for F-test: 3.2592


In [98]:
# degrees of freedom for block df_b = [b-1, (t-1)(b-1)] = (3, 12)
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'

f = np.abs(round(stats.f.isf(q = 0.05, dfn = 3, dfd = 12), 4))
print('Critical value (block - JobInvolvement) for F-test:', f)

Critical value (block - JobInvolvement) for F-test: 3.4903


In [96]:
# perform two-way ANOVA
test = ols('MonthlyIncome ~ Q("EducationField") + Q("JobInvolvement" )', sales_emp).fit()

# create table for 2-way ANOVA test
anova_2 = anova_lm(test, typ = 2)

# print the table
anova_2

,sum_sq,df,F,PR(>F)
"Q(""EducationField"")",3.661317e+07,4.0,0.822651,0.517502
"Q(""JobInvolvement"")",1.274268e+07,3.0,0.381749,0.766613
Residual,5.118225e+08,46.0,NaN,NaN


* The above output shows that the p-value for EducationField is greater than 0.05 
* The Test statistic (0.8226) for EducationField is less than Chi-crit (3.2592) i.e. it falls in acceptance region of H01
* Thus we fail to reject the null hypothesis (H01) for EducationField and conclude that the monthly income of employees does not depend on the education field


* The above output shows that the p-value for JobInvolvement is greater than 0.05 
* The Test statistic (0.3817) for JobInvolvement is less than Chi-crit (3.4903) i.e. it falls in acceptance region of H02
* Thus we fail to reject the null hypothesis (H02) for JobInvolvement and conclude that the monthly income of employees does not depend on the job involvement

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>20. Check whether the daily rate is affected by the types of business travel and levels of work-life balance. Use the p-value technique with 1% level of significance.</b>
                </font> 
            </div>
        </td>
    </tr>
</table>

In [99]:
sales_emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       54 non-null     int64 
 1   BusinessTravel            54 non-null     object
 2   DailyRate                 54 non-null     int64 
 3   DistanceFromHome          54 non-null     int64 
 4   Education                 54 non-null     int64 
 5   EducationField            54 non-null     object
 6   EmployeeCount             54 non-null     int64 
 7   EmployeeNumber            54 non-null     int64 
 8   EnvironmentSatisfaction   54 non-null     int64 
 9   HourlyRate                54 non-null     int64 
 10  JobInvolvement            54 non-null     object
 11  JobLevel                  54 non-null     int64 
 12  JobSatisfaction           54 non-null     int64 
 13  MaritalStatus             54 non-null     object
 14  MonthlyIncome             54

In [100]:
# since default Dtype of WorkLifeBalance is int64, we first need to make it categorical/nominal 
sales_emp['WorkLifeBalance'] = sales_emp['WorkLifeBalance'].astype(object)

The two-way ANOVA can be used to test the effect due to the two nominal variables on a numeric variable.

The null and alternative hypothesis for business travel:
* H01 : The daily rate is independent of business travel
* H11 : The daily rate is dependent on business travel

The null and alternative hypothesis for levels of work-life balance:
* H02 : The daily rate is independent of work-life balance
* H12 : The daily rate is dependent on work-life balance

In [101]:
# perform Shapiro-Wilk test to test the normality
stat, p_value = stats.shapiro(sales_emp['DailyRate'])

# print the p-value
print('p-value:', p_value)

p-value: 0.040250398218631744


From the above result, we can see that the p-value is greater than 0.01, thus we can say that the daily rate from different types of business travel and work-life balance are normally distributed. Thus the assumption of normality is satisfied.

In [102]:
# obtain number of unique education fields (treatments)
t = sales_emp['BusinessTravel'].nunique()
print('t:', t)

# obtain number of unique job involvement levels (blocks)
b = sales_emp['WorkLifeBalance'].nunique()
print('b:', b)

# consider the total number of employees in all education fields
N = sales_emp['BusinessTravel'].value_counts().sum()
print('N:', N)

t: 3
b: 4
N: 54


In [103]:
# degrees of freedom for treatment df_tr = [t-1, (t-1)(b-1)] = (2, 6)
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'

f = np.abs(round(stats.f.isf(q = 0.01, dfn = 2, dfd = 6), 4))
print('Critical value (treatment - BusinessTravel) for F-test:', f)

Critical value (treatment - BusinessTravel) for F-test: 10.9248


In [104]:
# degrees of freedom for block df_b = [b-1, (t-1)(b-1)] = (3, 6)
# calculate the F-value for 95% of confidence level
# use 'stats.f.isf()' to find the F-value corresponding to the upper tail probability 'q'

f = np.abs(round(stats.f.isf(q = 0.01, dfn = 3, dfd = 6), 4))
print('Critical value (block - WorkLifeBalance) for F-test:', f)

Critical value (block - WorkLifeBalance) for F-test: 9.7795


In [105]:
# perform two-way ANOVA
test = ols('DailyRate ~ Q("BusinessTravel") + Q("WorkLifeBalance" )', sales_emp).fit()

# create table for 2-way ANOVA test
anova_2 = anova_lm(test, typ = 2)

# print the table
anova_2

,sum_sq,df,F,PR(>F)
"Q(""BusinessTravel"")",3.559568e+06,2.0,14.293493,0.000013
"Q(""WorkLifeBalance"")",7.806979e+04,3.0,0.208993,0.889682
Residual,5.976821e+06,48.0,NaN,NaN


* The above output shows that the p-value for BusinessTravel is lesser than 0.01
* The Test statistic (14.2934) for BusinessTravel is greater than Chi-crit (10.9248) i.e. it falls in rejection region of H01
* Thus reject the null hypothesis (H01) for BusinessTravel and conclude that the daily rate of employees depends on the type of business travel


* The above output shows that the p-value for WorkLifeBalance is greater than 0.01 
* The Test statistic (0.2089) for WorkLifeBalance is less than Chi-crit (9.7795) i.e. it falls in acceptance region of H02
* Thus we fail to reject the null hypothesis (H02) for WorkLifeBalance and conclude that the daily rate of employees does not depend on the work-life balance